In [4]:

from pathlib import Path
from py2neo import Graph
from pandas import DataFrame
graph = Graph("bolt://localhost:7687")


In [36]:

df = DataFrame(graph.run("""
MATCH (snp:`gwas:SNP`)-[:`gwas:associated_with`]->(measurement)
MATCH (measurement)<-[:`upheno:phenotypeToTrait`]-(phenotype)
MATCH (phenotype)<-[:`biolink:broad_match`]-(descendantPhenotype)-[:sourceId]->(descendantSourceId)
WHERE "OLS.mp" IN descendantPhenotype.`grebi:datasources`
MATCH (descendantPhenotype)<-[:`impc:phenotype`]-(mouseGene)
WHERE "IMPC" IN mouseGene.`grebi:datasources`
RETURN DISTINCT [id in snp.id WHERE id =~ "rs[0-9]*" | id][0] AS gwas_variant,
[id in measurement.id WHERE id =~ "oba:.*" | id][0] AS measurement_id,
measurement.`grebi:name`[0] as measurement_name,
[id in phenotype.id WHERE id =~ "upheno:[0-9]*" | id][0] AS species_neutral_phenotype_id,
phenotype.`grebi:name`[0] AS species_neutral_phenotype_name,
[id in descendantPhenotype.id WHERE id =~ "mp:[0-9]*" | id][0] AS mouse_phenotype,
descendantPhenotype.`grebi:name`[0] AS mouse_phenotype_name,
mouseGene.`grebi:name`[0] AS mouse_gene_name,
[id in mouseGene.id WHERE id =~ "mgi:[0-9]*" | id][0] AS mouse_gene_id
""").data())

#print(df.head(5).to_markdown())
df.to_csv("gwas_to_impc.csv", index=False)

